In [1]:
import numpy as np 
import pandas as pd 
import geopandas as gpd 
import folium 
import datetime
import json
import branca
import statistics

In [2]:
data_stop = pd.read_csv("/home/thd/idf-accessibility/stop_ponctuels/200points/csv/merged1000SP_20h30_200points.csv", sep=";") #Use the right CSV (found in the ponctuel folder)
#merged "Name;Coord;Access;time\n"

In [3]:
Coord_gps_list = []

for coord in list(data_stop['Coord']):
    index = coord.find(',')
    lon = coord[:index]
    lat = coord[index+1:]
    Coord_gps_list.append([lon,lat])

data_stop['Coord_gps'] = Coord_gps_list
data_stop.head()

,Name,Coord,Access,time,Coord_gps
0,André Malraux,"48.77898,2.035979",3295.59,03-08-2021 08:30,"[48.77898, 2.035979]"
1,Jean Jaurès,"48.974702,1.803076",7716.23,03-08-2021 08:30,"[48.974702, 1.803076]"
2,Collège L. de Vincy,"48.463944,2.350624",17.54,03-08-2021 08:30,"[48.463944, 2.350624]"
3,GARE DE ST GRATIEN,"48.964243,2.285417",3502.30,03-08-2021 08:30,"[48.964243, 2.285417]"
4,MAISONS ALFORT ALFORTVILLE NOCTILIEN,"48.801627,2.427066",1948.35,03-08-2021 08:30,"[48.801627, 2.427066]"


In [4]:
len(data_stop[data_stop['Access'] == 0])
data_stop[data_stop['Access'] != 0]['Access'][5]

6340.81

In [16]:
#Change map's title
def visualization_map(N): #N : int such as the N% inferior and superior values are excluded

    coords = (48.8626304852,2.33629344655) #We center the map on Paris Ist
    map = folium.Map(location=coords, tiles='OpenStreetMap', zoom_start=9)
    if N != 0 :
        N = N/100
        list_quantiles = statistics.quantiles(data_stop[data_stop['Access'] != 0]['Access'], n=int(1/N), method='exclusive') #intervals de len(data['access]!=0)/N values 
        v_min = min(data_stop[(data_stop['Access'] != 0) & (data_stop['Access']>=list_quantiles[0])]['Access']) #min of the new dataset
        v_max = max(data_stop[(data_stop['Access'] != 0) & (data_stop['Access']<=list_quantiles[-1])]['Access']) #max of the new dataset
    else : 
        v_min = min(data_stop[data_stop['Access'] != 0]['Access'])
        v_max = max(data_stop['Access'])

    cm = branca.colormap.LinearColormap(['green', 'red'], vmin=v_min/v_max, vmax=v_max/v_max)
    cm.caption = 'Accessibility'+' '+data_stop['time'][0] #Color bar legend
    map.add_child(cm) 

    for i in range(len(data_stop)):
        if (list_quantiles[-1]>=data_stop['Access'][i]>=list_quantiles[0]): 
            folium.CircleMarker(
                location = (data_stop['Coord_gps'][i][0],data_stop['Coord_gps'][i][1]), #(data_stop['lat'][i], data_stop['lon'][i]), 
                popup = (data_stop['Name'][i],data_stop['Access'][i]/v_max), #Displays stop point name 
                radius = 1/(data_stop['Access'][i])*2*10e1, #Adjust regarding the result
                color = cm(data_stop['Access'][i]/v_max),  
                fill = True,
                fill_color = cm(data_stop['Access'][i]/v_max),
                fill_opacity=0.8
            ).add_to(map)
        elif (0<data_stop['Access'][i]<list_quantiles[0]) : 
            folium.CircleMarker(
                location = (data_stop['Coord_gps'][i][0],data_stop['Coord_gps'][i][1]), #(data_stop['lat'][i], data_stop['lon'][i]), 
                popup = (data_stop['Name'][i],data_stop['Access'][i]/v_max,'valeur trop petite'), #legende point
                radius = 2, 
                color = "blue",  #blue for values which access is in ]0,k[%
                fill = True,
                fill_color = "blue",
                fill_opacity=0.9
            ).add_to(map)
        elif (data_stop['Access'][i]>list_quantiles[-1]):
            folium.CircleMarker(
                location = (data_stop['Coord_gps'][i][0],data_stop['Coord_gps'][i][1]), #(data_stop['lat'][i], data_stop['lon'][i]), 
                popup = (data_stop['Name'][i],data_stop['Access'][i]/v_max,'valeur trop grande'), #legende point
                radius = 2, 
                color = "dark",  #black for values which access is in (100-k)% higher
                fill = True,
                fill_color = "dark",
                fill_opacity=0.9
            ).add_to(map)

    map.save(outfile='./200points/map/map_ponctuelle_1000SP_200_'+str(N*100)+'%'+'_8_mars'+data_stop['time'][0][data_stop['time'][0].find(' ')+1:]+'.html')
    return map


In [17]:
visualization_map(5)

In [13]:
#Dans ce code on distingue les points dont l'accessibilité est == 0 (en bleue) et on les exclu de la color bar

coords = (48.8626304852,2.33629344655) #on centre la map sur paris 1er
map = folium.Map(location=coords, tiles='OpenStreetMap', zoom_start=9)
cm = branca.colormap.LinearColormap(['green', 'red'], vmin=min(data_stop[data_stop['Access'] != 0]['Access'])/max(data_stop['Access']), vmax=max(data_stop['Access'])/max(data_stop['Access'])) 
cm.caption = 'Accessibility'+' '+data_stop['time'][0]
map.add_child(cm) 


for i in range(len(data_stop)):
    if (data_stop['Access'][i] != 0) : 
        folium.CircleMarker(
            location = (data_stop['Coord_gps'][i][0],data_stop['Coord_gps'][i][1]), #(data_stop['lat'][i], data_stop['lon'][i]), 
            popup = (data_stop['Name'][i],data_stop['Access'][i]/max(data_stop['Access'])), #affiche nom du stop 
            radius = 1/(data_stop['Access'][i])*500, #peut planter 
            color = cm(data_stop['Access'][i]/max(data_stop['Access'])),  
            fill = True,
            fill_color = cm(data_stop['Access'][i]/max(data_stop['Access'])),
            fill_opacity=0.8
        ).add_to(map)
    else : 
        folium.CircleMarker(
            location = (data_stop['Coord_gps'][i][0],data_stop['Coord_gps'][i][1]), #(data_stop['lat'][i], data_stop['lon'][i]), 
            popup = (data_stop['Name'][i],data_stop['Access'][i]/max(data_stop['Access'])), #affiche nom du sotp 
            radius = 1.5, #peut planter à voir /max
            color = "blue",  #bleue pour les valeurs dont l'accessbilité n'a pas été calculé
            fill = True,
            fill_color = "blue",
            fill_opacity=0.9
        ).add_to(map)

map.save(outfile='map_ponctuelle_test_100points'+'_'+data_stop['time'][0][data_stop['time'][0].find(' ')+1:]+'.html')
map